In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from PIL import Image
import skimage
from skimage import io, color
import tensorflow as tf
import cv2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [4]:

X_train_path = "/kaggle/input/original-data/original data/TrainDataAug/"
X_train_noise_path = "/kaggle/input/original-data/original data/TrainDataNoisyAug/"
X_test_path= "/kaggle/input/original-data/original data/TestDataAug/"
X_test_noisy_path="/kaggle/input/original-data/original data/TestDataNoisyAug/"


clean_images = []

for filename in os.listdir(X_train_path):
    if filename.endswith(".tif"):
        img = Image.open(X_train_path+filename).convert('L')
        img_array = np.array(img)
        clean_images.append(img_array)

clean_images = np.array(clean_images)
clean_images = clean_images.reshape(clean_images.shape + (1, ))


noisy_images = []

for filename in os.listdir(X_train_noise_path):
    if filename.endswith(".tif"):
        img = Image.open(X_train_noise_path+filename).convert('L')
        img_array = np.array(img)
        noisy_images.append(img_array)        

noisy_images = np.array(noisy_images)
noisy_images = noisy_images.reshape(noisy_images.shape + (1, ))


Testing_images=[]
for filename in os.listdir(X_test_path):
    if filename.endswith(".tif"):
        img = Image.open(X_test_path+filename).convert('L')
        img_array = np.array(img)
        Testing_images.append(img_array)

Testing_images= np.array(Testing_images)
Testing_images = Testing_images.reshape(Testing_images.shape + (1, ))


Testing_noisy_images=[]
for filename in os.listdir(X_test_noisy_path):
    if filename.endswith(".tif"):
        img = Image.open(X_test_noisy_path+filename).convert('L')
        img_array = np.array(img)
        Testing_noisy_images.append(img_array)

Testing_noisy_images= np.array(Testing_noisy_images)
Testing_noisy_images = Testing_noisy_images.reshape(Testing_noisy_images.shape + (1, ))

X_train = clean_images
X_train_noisy = noisy_images
X_test=Testing_images
X_test_noisy=Testing_noisy_images
X_train = np.repeat(X_train, 3, axis=3)
X_train_noisy = np.repeat(X_train_noisy, 3, axis=3)
X_test = np.repeat(X_test, 3, axis=3)
X_test_noisy = np.repeat(X_test_noisy, 3, axis=3)
X_train = X_train.astype('float32') /255
X_train_noisy = X_train_noisy.astype('float32') /255
X_test = X_test.astype('float32') /255
X_test_noisy = X_test_noisy.astype('float32') /255

In [5]:
pretrained_mobilenetv2 = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights=None)
pretrained_mobilenetv2.load_weights('/kaggle/input/mobilenet-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')
# Freeze the pre-trained layers
for layer in pretrained_mobilenetv2.layers:
    layer.trainable = False

# Add new layers to decode the image to 128x128
x = pretrained_mobilenetv2.output
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Define the model
model = Model(inputs=pretrained_mobilenetv2.input, outputs=decoded)
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='mse')
# Print summary of the model
model.summary( )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
# Create an instance of EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=1, verbose=1)
history=model.fit(X_train_noisy,X_train,
                epochs=36,
                batch_size=32,
                shuffle=True,
                validation_split=0.1,callbacks=[early_stopping])
model.save("mobilenetv2_trained.h5")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train loss', 'val loss'], loc='upper right')
plt.show()

In [ ]:
loss= model.evaluate(X_test_noisy, X_test)
print('Test loss:', loss) 

In [ ]:
def show_predicted_images(images, labels):
    fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15, 15))

    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        ax.set_title("Predicted Label: {}")
        ax.axis('off')

    plt.show()


In [ ]:
predicted_images = model.predict(X_test_noisy)
predicted_labels = np.argmax(predicted_images, axis=1)
show_predicted_images(predicted_images, predicted_labels)

